# Run MPI

In [1]:
!mpirun -n 4 python ../../examples/mpi_examples/mpi_calc_molsurf_0_single_traj.py

('total array len: ', 1000)


In [2]:
!mpirun -n 5 python ../../examples/mpi_examples/mpi_calc_molsurf_0_list_of_trajs.py

('total array len: ', 5000)


In [3]:
import mpi4py
from pytraj.__version__ import __version__ as pytraj_version
print (mpi4py.__version__)
print (pytraj_version)

1.3
0.1.2.dev4


# Code example: Single trajectory

In [ ]:
# %load ../../examples/mpi_examples/mpi_calc_molsurf_0_single_traj.py
# (require: mpi4py, numpy)
# mpirun -n 4 python mpi_cal_molsurf_0.py

# always add those lines to your code
import numpy as np
from mpi4py import MPI
from pytraj.parallel import map as pymap
from pytraj import io
import pytraj.common_actions as pyca

comm = MPI.COMM_WORLD
# end. you are free to update anything below here

# split remd.x.000 to N cores and do calc_surf in parallel
root_dir = "../../tests/data/nogit/remd/"
traj_name = root_dir + "/remd.x.000"
parm_name = root_dir + "myparm.top"

# load to TrajReadOnly
traj = io.load(traj_name, parm_name)

# mapping different chunk of `traj` in N cores
# need to provide `comm`
# save `total_arr` to rank=0
# others: total_arr = None
total_arr = pymap(comm, pyca.calc_molsurf, traj, "@CA", top=traj.top, root=0)

if comm.rank != 0:
    assert total_arr is None

if comm.rank == 0:
    # skip final array since its shape might be different from the rest
    t0 = np.asarray(total_arr[:-1]).flatten()
    t1 = np.asarray(total_arr[-1]).flatten()
    t = np.append(t0, t1)
    print ('total array len: ', t.shape[0])

    # assert to serial values
    #t2 = pyca.calc_molsurf(traj, "@CA", dtype='ndarray')
    #assert t.shape == t2.shape
    #assert np.any(t == t2) == True


# Code example: a list of trajectory

In [ ]:
# %load ../../examples/mpi_examples/mpi_calc_molsurf_0_list_of_trajs.py
# (require: mpi4py, numpy)
# mpirun -n 4 python mpi_cal_molsurf_0.py

# always add those lines to your code
import numpy as np
from mpi4py import MPI
from pytraj.parallel import map as pymap
from pytraj import io
import pytraj.common_actions as pyca

comm = MPI.COMM_WORLD
# end. you are free to update anything below here

# split remd.x.000 to N cores and do calc_molsurf in parallel
root_dir = "../../tests/data/nogit/remd/"
parm_name = root_dir + "myparm.top"
top = io.load(parm_name)

# load to TrajReadOnly
trajlist = []
for i in range(comm.size):
    ext = "00" + str(i)
    traj_name = root_dir + "/remd.x." + ext # 000, 001, 002
    trajlist.append(io.load(traj_name, top))

# mapping different traj to N cores
# need to provide `comm`
total_arr = pymap(comm, pyca.calc_molsurf, trajlist, "@CA", top=top)

# gathering the data to root=0
#if comm.rank == 0:
#    total_arr =  np.empty(comm.size)
#else:
#    total_arr = None
#total_arr = comm.gather(arr, root=0)

if comm.rank == 0:
    # skip final array since its shape might be different from the rest
    t0 = np.asarray(total_arr[:-1]).flatten()
    t1 = np.asarray(total_arr[-1]).flatten()
    t = np.append(t0, t1)
    print ('total array len: ', t.shape[0])

    # assert to serial values
    #t2 = pyca.calc_molsurf(trajlist, "@CA", dtype='ndarray')
    #assert t.shape == t2.shape
    #assert np.any(t == t2) == True
